In [ ]:
import json
import pandas as pd
import requests
import sqlalchemy
from datetime import datetime, timedelta, date


def get_flights(cities_df, airports_df):
    
    city_list = cities_df["city_name"]

    # creating empty icaos list for API input
    icaos_list = []
    icaos_flat_list = []

    # filling icao list
    for city in city_list:
        icaos_list.append(airports_df.loc[airports_df["city_name"]==city]["airport_icao"].to_list())

    icaos_flat_list = [item for sublist in icaos_list for item in sublist]

    # Get Tomorrow
    tomorrow = datetime.now() + timedelta(1)
    tomorrow_url = tomorrow.strftime("%Y-%m-%d")

    # creating empty list for eventual dataframe
    list_for_df = []

    # iterating through icao list to get flights info for 24 hours instead of just 12
    for icao in icaos_flat_list:

        for i in range(2):

            url =  [f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow_url}T00:00/{tomorrow_url}T12:00",
                    f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow_url}T12:01/{tomorrow_url}T23:59"
                    ]    

            querystring = {"withLeg":"true"}

            headers = {
                "X-RapidAPI-Key": "",
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }

            response = requests.get(url[i], headers=headers, params=querystring)
            
            # normalize JSON file and access 'arrivals' directory
            arrivals = pd.json_normalize(response.json()["arrivals"])

            # creating dataframe with normalized JSON dataframe
            arrivals_df = pd.DataFrame({"flight_num": arrivals.loc[:,"number"],
                            "departure_icao": arrivals.loc[:,"departure.airport.icao"],
                            "arrival_icao": icao,
                            "arrival_time": arrivals.loc[:,"arrival.scheduledTime.local"].str[:-6].astype(str) + ":00"
                            })
            arrivals_df["arrival_time"] = pd.to_datetime(arrivals_df["arrival_time"])
                        
            list_for_df.append(arrivals_df)
    
    # adding dataframe to pre-existing ones
    flights_df = pd.concat(list_for_df, ignore_index=True)       

    return flights_df

def lambda_handler(event, context):
    # setting variables for sql connection
    schema="gans"
    host=""
    user="admin"
    password=""
    port=3306
    con = f"mysql+pymysql://{user}:{password}@{host}:{port}/{schema}"
    
    # reading cities and airports table
    cities_sql = pd.read_sql_table("cities", con=con)
    print(cities_sql)
    airports_sql = pd.read_sql_table("airports", con=con)
    print(airports_sql)
    # executing Flight API call function
    get_flights(cities_sql, airports_sql).to_sql("flights", if_exists="append", con=con, index=False)
    
    return {
    'statusCode': 200,
    'body': json.dumps('Hello from Lambda!')}